In [2]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.5.0`

import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
  .appName("ScalaSpark")
  .master("local[*]")
  .getOrCreate()

import spark.implicits._


25/03/30 15:27:12 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
25/03/30 15:27:12 INFO SharedState: Warehouse path is 'file:/home/eyal/code/notebooks/spark-warehouse'.


import $ivy.$
import org.apache.spark.sql.SparkSession
spark: SparkSession = org.apache.spark.sql.SparkSession@6e3a128b
import spark.implicits._

In [4]:
import org.apache.spark.sql.expressions.Aggregator
import org.apache.spark.sql.Encoders
import org.apache.spark.sql.functions.udaf

case class IntSumBuffer(var sum: Long = 0L)

object SumAll extends Aggregator[Int, IntSumBuffer, Long] {
  def zero = IntSumBuffer(0L)
  def reduce(buffer: IntSumBuffer, value: Int): IntSumBuffer = {
    buffer.sum += value
    buffer
  }
  def merge(b1: IntSumBuffer, b2: IntSumBuffer): IntSumBuffer = {
    b1.sum += b2.sum
    b1
  }
  def finish(reduction: IntSumBuffer): Long = reduction.sum
  def bufferEncoder = Encoders.product[IntSumBuffer]
  def outputEncoder = Encoders.scalaLong
}

val sumAll = udaf(SumAll)


import org.apache.spark.sql.expressions.Aggregator
import org.apache.spark.sql.Encoders
import org.apache.spark.sql.functions.udaf
defined class IntSumBuffer
defined object SumAll
sumAll: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedAggregator(
  aggregator = ammonite.$sess.cmd4$Helper$SumAll$@25d55b21,
  inputEncoder = ExpressionEncoder(
    objSerializer = BoundReference(
      ordinal = 0,
      dataType = IntegerType,
      nullable = false
    ),
    objDeserializer = AssertNotNull(
      child = UpCast(
        child = GetColumnByOrdinal(ordinal = 0, dataType = IntegerType),
        target = IntegerType,
        walkedTypePath = List("- root class: \"int\"")
      ),
      walkedTypePath = List("- root class: \"int\"")
    ),
    clsTag = Int
  ),
  name = None,
  nullable = true,
  deterministic = true
)

In [5]:
val df = Seq(1, 2, 3, 4, 5).toDF("value")
df.select(sumAll($"value").as("total_sum")).show()

25/03/30 15:29:11 INFO DAGScheduler: Registering RDD 3 (show at cmd5.sc:2) as input to shuffle 0
25/03/30 15:29:11 INFO DAGScheduler: Got map stage job 0 (show at cmd5.sc:2) with 5 output partitions
25/03/30 15:29:11 INFO DAGScheduler: Final stage: ShuffleMapStage 0 (show at cmd5.sc:2)
25/03/30 15:29:11 INFO DAGScheduler: Parents of final stage: List()
25/03/30 15:29:11 INFO DAGScheduler: Missing parents: List()
25/03/30 15:29:11 INFO DAGScheduler: Submitting ShuffleMapStage 0 (MapPartitionsRDD[3] at show at cmd5.sc:2), which has no missing parents
25/03/30 15:29:11 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 21.3 KiB, free 2.2 GiB)
25/03/30 15:29:11 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 9.3 KiB, free 2.2 GiB)
25/03/30 15:29:11 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on 10.255.255.254:34797 (size: 9.3 KiB, free: 2.2 GiB)
25/03/30 15:29:11 INFO SparkContext: Created broadcast 0 from broad

+---------+
|total_sum|
+---------+
|       15|
+---------+



df: org.apache.spark.sql.package.DataFrame = [value: int]